<a href="https://colab.research.google.com/github/yanghyeon408/Dacon/blob/master/RandomForest_8m.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df=pd.read_csv('201901-202003.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24697792 entries, 0 to 24697791
Data columns (total 12 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   REG_YYMM      int64 
 1   CARD_SIDO_NM  object
 2   CARD_CCG_NM   object
 3   STD_CLSS_NM   object
 4   HOM_SIDO_NM   object
 5   HOM_CCG_NM    object
 6   AGE           object
 7   SEX_CTGO_CD   int64 
 8   FLC           int64 
 9   CSTMR_CNT     int64 
 10  AMT           int64 
 11  CNT           int64 
dtypes: int64(6), object(6)
memory usage: 2.2+ GB


In [ ]:
def grap_year(data):
    data = str(data)
    return int(data[:4])

def grap_month(data):
    data = str(data)
    return int(data[4:])

In [ ]:
# 날짜 처리

data = df.fillna('')
data['year'] = data['REG_YYMM'].apply(lambda x: grap_year(x))
data['month'] = data['REG_YYMM'].apply(lambda x: grap_month(x))
data = data.drop(['REG_YYMM'], axis=1)

In [ ]:
# 데이터 정제
df = data.copy()
df = df.drop(['CARD_CCG_NM', 'HOM_CCG_NM'], axis=1)

In [ ]:
columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month']
df = df.groupby(columns).sum().reset_index(drop=False)

In [ ]:
# 인코딩
dtypes = df.dtypes
encoders = {}
for column in df.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(df[column])
        encoders[column] = encoder
        
df_num = df.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    df_num[column] = encoder.transform(df[column])

In [ ]:
df = df_num[['year', 'month', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'CNT', 'AMT']]

In [ ]:
df.head()

,year,month,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,CNT,AMT
0,2019,1,0,0,0,1,1,4,311200
1,2019,2,0,0,0,1,1,3,605000
2,2019,6,0,0,0,1,1,3,139000
3,2019,8,0,0,0,1,1,3,27500
4,2019,9,0,0,0,1,1,3,395500


In [ ]:
df.drop(df.loc[df['month']==12].index, inplace=True)

/Users/yanghyeonkim/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
df.drop(df.loc[df['month']==11].index, inplace=True)

In [ ]:
df.drop(df.loc[df['month']==10].index, inplace=True)

In [ ]:
df.drop(df.loc[df['month']==9].index, inplace=True)

/Users/yanghyeonkim/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
df.shape

(767489, 9)

In [ ]:
# feature, target 설정
train_num = df.sample(frac=1, random_state=0)
train_features = train_num.drop(['AMT', 'CNT'], axis=1)
train_target = np.log1p(train_num['AMT'])

In [ ]:
# 훈련
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_jobs=-1, random_state=0)
model.fit(train_features, train_target)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [ ]:
# 예측 템플릿 만들기
CARD_SIDO_NMs = df['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = df['STD_CLSS_NM'].unique()
HOM_SIDO_NMs  = df['HOM_SIDO_NM'].unique()
AGEs          = df['AGE'].unique()
SEX_CTGO_CDs  = df['SEX_CTGO_CD'].unique()
years         = [2020]
months        = [4, 7]

temp = []
for year in years:
    for month in months:
        for CARD_SIDO_NM in CARD_SIDO_NMs:
            for STD_CLSS_NM in STD_CLSS_NMs:
                for HOM_SIDO_NM in HOM_SIDO_NMs:
                    for AGE in AGEs:
                        for SEX_CTGO_CD in SEX_CTGO_CDs:
                            temp.append([year, month, CARD_SIDO_NM, STD_CLSS_NM, HOM_SIDO_NM, AGE, SEX_CTGO_CD])
temp = np.array(temp)
temp = pd.DataFrame(data=temp, columns=train_features.columns)

In [ ]:
temp

,year,month,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD
0,2020,4,0,0,0,1,1
1,2020,4,0,0,0,1,2
2,2020,4,0,0,0,2,1
3,2020,4,0,0,0,2,2
4,2020,4,0,0,0,3,1
...,...,...,...,...,...,...,...
331767,2020,7,16,30,14,5,2
331768,2020,7,16,30,14,6,1
331769,2020,7,16,30,14,6,2
331770,2020,7,16,30,14,0,1


In [ ]:
# 예측
pred = model.predict(temp)
pred = np.expm1(pred)
temp['AMT'] = np.round(pred, 0)
temp['REG_YYMM'] = temp['year']*100 + temp['month']
temp = temp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

In [ ]:
# 디코딩
import sklearn
from sklearn.preprocessing import LabelEncoder

temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

In [ ]:
sub = pd.read_csv('submission.csv', index_col=0)
sub = sub.drop(['AMT'], axis=1)
sub = sub.merge(temp, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
sub.index.name = 'id'
sub.to_csv('submission_1_8m.csv', encoding='utf-8-sig')
sub.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,1.290193e+08
1,202004,강원,골프장 운영업,1.788795e+09
2,202004,강원,과실 및 채소 소매업,6.516155e+08
3,202004,강원,관광 민예품 및 선물용품 소매업,2.821356e+07
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,2.656095e+07
